In [83]:
from collections import defaultdict
import numpy as np, pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
import json
import torch

In [84]:
!pip install transformers

Error initializing plugin EntryPoint(name='Windows (alt)', value='keyrings.alt.Windows', group='keyring.backends').
Traceback (most recent call last):
  File "/home/postyakov/.local/lib/python3.6/site-packages/keyring/backend.py", line 203, in _load_plugins
    init_func = ep.load()
  File "/usr/local/lib/python3.6/dist-packages/importlib_metadata/__init__.py", line 96, in load
    module = import_module(match.group('module'))
  File "/usr/lib/python3.6/importlib/__init__.py", line 126, in import_module
    return _bootstrap._gcd_import(name[level:], package, level)
  File "<frozen importlib._bootstrap>", line 994, in _gcd_import
  File "<frozen importlib._bootstrap>", line 971, in _find_and_load
  File "<frozen importlib._bootstrap>", line 955, in _find_and_load_unlocked
  File "<frozen importlib._bootstrap>", line 665, in _load_unlocked
  File "<frozen importlib._bootstrap_external>", line 678, in exec_module
  File "<frozen importlib._bootstrap>", line 219, in _call_with_frames_remo

In [85]:
import pandas as pd

In [86]:
with open('res_cor.json') as data:
    file = json.load(data)

In [87]:
dialogues = [] 
for d in file[:2]:
    samples = defaultdict(dict)
    result = d['completions'][0]['result']
    texts_without_labels = d['data']['text']
    for sample in result:
        speaker = texts_without_labels[int(sample['value']['start'])]['speaker']
        samples[sample['id']]['speaker'] = speaker
        samples[sample['id']]['text'] = sample['value']['text']
        samples[sample['id']]['start'] = int(sample['value']['start'])
        if 'paragraphlabels' in sample['value']:
            samples[sample['id']]['paragraphlabels'] = sample['value']['paragraphlabels'][0]
        if 'choices' in sample['value']:
            samples[sample['id']]['choices'] = sample['value']['choices'][0]
    
    sorted_samples = sorted([(samples[sample_id]['start'], sample_id) for sample_id in samples])
    texts = []
    labels = []
    speakers = []
    for _, sample_id in sorted_samples:
        if samples[sample_id]['text'] != 'PAUSE':
            texts.append(str(samples[sample_id]['text']).replace('\n', ''))
            speakers.append(samples[sample_id]['speaker'])
            paragraph_labels = samples[sample_id].get('paragraphlabels', '')
            choices = samples[sample_id].get('choices', '')
#         if not choices:
#             print(sample_id)
            labels.append(paragraph_labels + '.' + choices)
    dialogues.append((texts, labels, speakers))


In [88]:
train_data = dialogues[1][0]
test_data = dialogues[0][0]

In [89]:
len(test_data)

460

In [90]:
train_labels = dialogues[1][1][1:]
test_labels = dialogues[0][1][1:]

In [91]:
len(test_labels)

459

In [92]:
train_speakers = dialogues[1][2]
test_speakers = dialogues[0][2]

In [93]:
from transformers import AutoTokenizer, AutoModel

tokenizer = AutoTokenizer.from_pretrained("DeepPavlov/bert-base-cased-conversational")
model = AutoModel.from_pretrained("DeepPavlov/bert-base-cased-conversational")

train_outputs = []
for text in train_data:
    with torch.no_grad():
        input_ph= tokenizer(str(text).replace('?uh', ''), padding=True, truncation=True, max_length=30,return_tensors="pt")
        output_ph = model(**input_ph)
#         train_outputs.append(output_ph.pooler_output.cpu().numpy()) 
        sentence_embedding = output_ph.last_hidden_state.mean(dim=1).cpu().numpy()
    train_outputs.append(sentence_embedding)

In [94]:
test_outputs = []
for text in test_data:
    with torch.no_grad():
        input_ph= tokenizer(str(text).replace('?uh', ''), padding=True, truncation=True, max_length=30,return_tensors="pt")
        output_ph = model(**input_ph)
#        train_outputs.append(output_ph.pooler_output.cpu().numpy()) 
        sentence_embedding = output_ph.last_hidden_state.mean(dim=1).cpu().numpy()
    test_outputs.append(sentence_embedding)

In [95]:
test_outputs = np.concatenate(test_outputs)
test_concatenated = np.concatenate([test_outputs[1:],test_outputs[:-1]], axis=1)

In [96]:
train_outputs = np.concatenate(train_outputs)

In [97]:
train_concatenated = np.concatenate([train_outputs[1:],train_outputs[:-1]], axis=1)

In [77]:
train_concatenated.shape

(460, 1536)

In [78]:
train_concatenated

array([[ 0.43688995,  0.19137517, -0.0198823 , ...,  0.8330983 ,
         0.31290597, -0.03456344],
       [ 0.05688643, -0.25340334, -0.44757494, ..., -0.25558057,
         0.3885941 , -0.24909207],
       [ 0.24975166,  0.29351747, -0.54521376, ...,  0.54846525,
         0.07448874, -0.2684228 ],
       ...,
       [ 0.5119918 , -0.64941543,  0.1267632 , ..., -0.22897026,
         0.54134154, -0.15236306],
       [ 0.14594515, -0.0383253 ,  0.2194182 , ...,  0.42088524,
         0.27806434,  0.33803844],
       [ 0.4970232 , -0.10763296, -0.06794327, ..., -0.41754875,
         0.48466605, -0.27968317]], dtype=float32)

In [79]:
from sklearn.linear_model import LogisticRegression 
model = LogisticRegression(C=0.1)
model.fit(train_concatenated,train_labels)
y_pred = model.predict(test_concatenated)

/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:469: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)
